In [63]:
import pandas as pd
import numpy as np
import selenium
import sqlalchemy
import pickle
import pymysql
import sqlalchemy
from urllib import parse
from tqdm import tqdm

In [43]:
sido_convert = {'서울특별시': ['서울', '서울시'], 
                '부산광역시': ['부산', '부산시'], 
                '대구광역시': ['대구', '대구시'], 
                '인천광역시': ['인천', '인천시'],
                '광주광역시': ['광주', '광주시'], 
                '대전광역시': ['대전', '대전시'], 
                '울산광역시': ['울산', '울산시'], 
                '세종특별자치시': ['세종', '세종시', '세종특별시'], 
                '경기도': ['경기'], 
                '강원특별자치도': ['강원', '강원도'], 
                '충청북도': ['충북'], 
                '충청남도': ['충남'], 
                '전북특별자치도': ['전북', '전라북도'], 
                '전라남도': ['전남'], 
                '경상북도': ['경북'], 
                '경상남도': ['경남'], 
                '제주특별자치도': ['제주도', '제주']}

In [111]:
user = 'encore4'
password = 'EnCore4!@#'
host='13.125.65.37'
port = 3306
database = 'encore4'
password = parse.quote_plus(password)
engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")

In [112]:
con =pymysql.connect(host='13.125.65.37', user='encore4', password='EnCore4!@#', db='encore4', charset='utf8')
cur = con.cursor()

 - - -
- car_regitrection_count_df: 전국 차량 등록현황

In [49]:
with open('./../../../data/dataset_miniproject1/car_regitrection_count.pkl', 'rb') as f:
    car_regitrection_count_df = pickle.load(f)

car_regitrection_count_df.head(1)

,년도,월,시도명,시군구,승용_관용,승용.1_자가용,승용.2_영업용,승용.3_계,승합_관용,승합.1_자가용,...,화물.2_영업용,화물.3_계,특수_관용,특수.1_자가용,특수.2_영업용,특수.3_계,총계_관용,총계.1_자가용,총계.2_영업용,총계.3_계
0,2011,01,서울,강남구,163,192610,31456,224229,86,8261,...,1630,19198,21,94,134,249,431,218372,34948,253751


In [50]:
print(car_regitrection_count_df.columns.tolist(), end='')
car_regitrection_count_new_order = ['년도', '월', '시도명', '시군구', '총계.3_계']

['년도', '월', '시도명', '시군구', '승용_관용', '승용.1_자가용', '승용.2_영업용', '승용.3_계', '승합_관용', '승합.1_자가용', '승합.2_영업용', '승합.3_계', '화물_관용', '화물.1_자가용', '화물.2_영업용', '화물.3_계', '특수_관용', '특수.1_자가용', '특수.2_영업용', '특수.3_계', '총계_관용', '총계.1_자가용', '총계.2_영업용', '총계.3_계']

In [51]:
car_regitrection_count_df_final = car_regitrection_count_df[car_regitrection_count_new_order].copy()

In [52]:
car_regitrection_count_new_order = ['year', 'month', 'sido', 'sigungu', 'total_count']
car_regitrection_count_df_final.columns = car_regitrection_count_new_order

In [53]:
for key, values in sido_convert.items():
    for v in values:
        car_regitrection_count_df_final.loc[car_regitrection_count_df_final.sido == v, 'sido'] = key

In [54]:
car_regitrection_count_df_final.sido.unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '경기도', '강원특별자치도', '충청북도', '충청남도', '전북특별자치도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도', '세종특별자치시'], dtype=object)

In [55]:
car_regitrection_count_df_final.sigungu.apply(lambda x: len(x)).max()

9

In [56]:
car_regitrection_count_df_final.head(1)

,year,month,sido,sigungu,total_count
0,2011,01,서울특별시,강남구,253751


In [57]:
car_regitrection_count_df_final.shape

(40008, 5)

In [137]:
sql = "SHOW TABLES"
cur.execute(sql)

10

In [58]:
sql = "INSERT INTO carRegitrectionCount VALUES (%s, %s, %s, %s, %s)"
car_regitrection_count_df_final.apply(lambda x : cur.execute(sql, x.tolist() ), axis=1)

0        1
1        1
2        1
3        1
4        1
        ..
40003    1
40004    1
40005    1
40006    1
40007    1
Length: 40008, dtype: int64

In [59]:
con.commit()

- - -
- charge_station_info: 전기 자동차 충전소 정보

In [22]:
with open('./../../../data/dataset_miniproject1/charge_station_info.pkl', 'rb') as f:
    charge_station_info_df = pickle.load(f)

charge_station_info_df.head(1)

,metro,city,stnPlace,stnAddr,rapidChargerCount,slowChargerCount,carType,coordinates
0,서울특별시,강남구,LH강남힐스테이트,서울특별시 강남구 자곡동 자곡로3길 21,1,4,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...","[127.0884362, 37.4711594]"


In [23]:
charge_station_info_df.rename(columns={'metro': 'sido', 'city': 'sigungu', 'stnAddr': 'address', 'stnPlace': 'station_name'}, inplace=True)

In [24]:
for key, values in sido_convert.items():
    for v in values:
        charge_station_info_df.loc[charge_station_info_df.sido == v, 'sido'] = key

In [25]:
charge_station_info_df.sido.unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '경기도', '강원특별자치도', '충청남도', '충청북도', '전북특별자치도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도'], dtype=object)

In [26]:
charge_station_info_df.head(1)

,sido,sigungu,station_name,address,rapidChargerCount,slowChargerCount,carType,coordinates
0,서울특별시,강남구,LH강남힐스테이트,서울특별시 강남구 자곡동 자곡로3길 21,1,4,"SM3 Z.E,레이EV,소울EV,닛산리프,아이오닉EV,BMW i3,스파크EV,볼트E...","[127.0884362, 37.4711594]"


In [27]:
charge_station_info_df['lng'] = charge_station_info_df.coordinates.apply(lambda x: x[0])
charge_station_info_df['lat'] = charge_station_info_df.coordinates.apply(lambda x: x[1])

In [28]:
charge_station_info_df.drop(labels='coordinates', axis = 1, inplace=True)

In [29]:
print(charge_station_info_df.columns.tolist(), end='')

['sido', 'sigungu', 'station_name', 'address', 'rapidChargerCount', 'slowChargerCount', 'carType', 'lng', 'lat']

In [30]:
charge_station_info_df.shape

(4594, 9)

In [42]:
cur.close()

In [37]:
charge_station_info_df[charge_station_info_df.address.apply(lambda x: len(x)>50)]

,sido,sigungu,station_name,address,rapidChargerCount,slowChargerCount,carType,lng,lat
2969,충청남도,보령시,농업기술센터,충청남도 null null null null null null null null n...,1,0,"SM3 Z.E,테슬라",0,0


In [40]:
charge_station_info_df.drop(2969, inplace=True)

In [46]:
sql = "INSERT INTO chargeStationInfo VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
charge_station_info_df.apply(lambda x : cur.execute(sql, x.tolist() ), axis=1)

0       1
1       1
2       1
3       1
4       1
       ..
4589    1
4590    1
4591    1
4592    1
4593    1
Length: 4593, dtype: int64

In [48]:
charge_station_info_df.shape

(4593, 9)

In [47]:
con.commit()

- - -
- gas_station_info_df: 전국 주유소 지리 정보

In [61]:
with open('./../../../data/dataset_miniproject1/gas_station_final_with_coor.pkl', 'rb') as f:
    gas_station_info_df = pickle.load(f)

gas_station_info_df.head(1)

,sido,sigungu,station_name,address,company,phone_number,coordinates
0,서울특별시,강남구,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),SK에너지,02-445-6870,"[127.1187331, 37.4672785]"


In [62]:
gas_station_info_df.shape

(10766, 7)

In [65]:
tqdm.pandas()
gas_station_info_df['lng'] = gas_station_info_df.coordinates.progress_apply(lambda x: x[0])
gas_station_info_df['lat'] = gas_station_info_df.coordinates.progress_apply(lambda x: x[1])

100%|██████████| 10766/10766 [00:00<00:00, 867881.55it/s]


In [68]:
gas_station_info_df.drop(columns='coordinates', axis=1, inplace=True)

In [69]:
gas_station_info_df

,sido,sigungu,station_name,address,company,phone_number,lng,lat
0,서울특별시,강남구,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),SK에너지,02-445-6870,127.1187331,37.4672785
1,서울특별시,강남구,HD현대오일뱅크㈜직영 산성셀프주유소,서울 강남구 헌릉로 730,HD현대오일뱅크,02-2226-4963,127.1190231,37.4665975
2,서울특별시,강남구,자곡셀프주유소,서울 강남구 밤고개로 120 (자곡동),SK에너지,02-445-5841,127.1040159,37.4835419
3,서울특별시,강남구,방죽주유소,서울 강남구 밤고개로 215 (율현동),GS칼텍스,02-459-3434,127.1068180,37.4749811
4,서울특별시,강남구,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),SK에너지,02-3462-5100,127.0395832,37.4858704
...,...,...,...,...,...,...,...,...
12254,제주특별자치도,제주시,열린주유소,제주 제주시 한북로 221 (오등동),SK에너지,064-702-1133,126.5386827,33.4603089
12255,제주특별자치도,제주시,신도시주유소,제주 제주시 과원로 88 (연동),SK에너지,064-746-5185,126.4921803,33.4777589
12256,제주특별자치도,제주시,인양주유소,제주 제주시 추자면 영흥4길 1,자가상표,064-742-3529,126.3002433,33.9572990
12257,제주특별자치도,제주시,삼광주유소,제주특별자치도 제주시 번영로 158 (화북이동),GS칼텍스,064-725-5188,126.5622421,33.5043342


In [70]:
for i in gas_station_info_df.columns:
    print(f'`{i}` ,')

`sido` ,
`sigungu` ,
`station_name` ,
`address` ,
`company` ,
`phone_number` ,
`lng` ,
`lat` ,


In [71]:
sql = "INSERT INTO gasStationInfo VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
gas_station_info_df.apply(lambda x : cur.execute(sql, x.tolist() ), axis=1)

0        1
1        1
2        1
3        1
4        1
        ..
12254    1
12255    1
12256    1
12257    1
12258    1
Length: 10766, dtype: int64

In [72]:
con.commit()

In [73]:
gas_station_info_df.shape

(10766, 8)

- - -
- highway_rest_station_df: 고속도록 휴게소 지리 정보

In [74]:
with open('./../../../data/dataset_miniproject1/highway_rest_station.pkl', 'rb') as f:
    highway_rest_station_df = pickle.load(f)

highway_rest_station_df.head(1)

,unitName,pageNo,numOfRows,unitCode,stdRestCd,routeNo,routeName,xValue,yValue,serviceAreaCode
0,서울만남(부산)휴게소,None,None,001,000001,0010,경부선,127.042514,37.459939,A00001


In [75]:
print(highway_rest_station_df.columns.tolist(), end='')

['unitName', 'pageNo', 'numOfRows', 'unitCode', 'stdRestCd', 'routeNo', 'routeName', 'xValue', 'yValue', 'serviceAreaCode']

In [76]:
rest_station_columns = ['unitName', 'routeName', 'xValue', 'yValue']
highway_rest_station_df_final = highway_rest_station_df[rest_station_columns].copy()

In [77]:
highway_rest_station_df_final.rename(columns={'unitName': 'rest_station_name', 'routeName': 'route_name', 'xValue': 'lng', 'yValue': 'lat'}, inplace=True)

In [78]:
for i in highway_rest_station_df_final.columns:
    print(f'`{i}` ,')

`rest_station_name` ,
`route_name` ,
`lng` ,
`lat` ,


In [79]:
highway_rest_station_df_final.route_name.unique()

array(['경부선', '남해선(순천-부산)', '광주대구선', '무안광주선', '서해안선', '호남선', '순천완주선',
       '청주영덕선', '통영대전선', '중부선', '평택제천선', '중부내륙선', '영동선', '중앙선', '서울양양선',
       '동해선(삼척-속초)', '동해선(부산-포항)', '수도권제1순환선', '남해2지선', '서천공주선', '호남지선',
       '중부내륙지선'], dtype=object)

In [80]:
highway_rest_station_df_final.head(1)

,rest_station_name,route_name,lng,lat
0,서울만남(부산)휴게소,경부선,127.042514,37.459939


In [81]:
sql = "INSERT INTO highwayRestStation VALUES (%s, %s, %s, %s)"
highway_rest_station_df_final.apply(lambda x : cur.execute(sql, x.tolist() ), axis=1)

0    1
1    1
2    1
3    1
4    1
    ..
5    1
6    1
7    1
8    1
9    1
Length: 210, dtype: int64

In [82]:
con.commit()

- - -
- pop_info_df: 2023년도 총인구조사

In [83]:
with open('./../../../data/dataset_miniproject1/pop_info.pkl', 'rb') as f:
    pop_info_df = pickle.load(f)
    
pop_info_df.head(5)

,행정구역,시,도,2023년_총인구수,2023년_세대수,2023년_세대당 인구
0,서울특별시,서울특별시,합계,9386034,4469417,2.10
1,서울특별시 종로구,서울특별시,종로구,139417,72067,1.93
2,서울특별시 중구,서울특별시,중구,121312,64714,1.87
3,서울특별시 용산구,서울특별시,용산구,213151,107825,1.98
4,서울특별시 성동구,서울특별시,성동구,277361,133089,2.08


In [84]:
print(pop_info_df.columns.tolist(), end='')

['행정구역', '시', '도', '2023년_총인구수', '2023년_세대수', '2023년_세대당 인구']

In [85]:
pop_info_cols = ['시', '도', '행정구역', '2023년_총인구수', '2023년_세대수', '2023년_세대당 인구']
pop_info_df_final = pop_info_df[pop_info_cols].copy()

In [86]:
pop_info_df_final.columns = ['sido', 'sigungu', 'address', 'total_pop', 'total_family', 'pop_per_family']

In [87]:
pop_info_df_final.loc[pop_info_df_final.sigungu.str.contains('합계')]

,sido,sigungu,address,total_pop,total_family,pop_per_family
0,서울특별시,합계,서울특별시,9386034,4469417,2.10
26,부산광역시,합계,부산광역시,3293362,1564588,2.10
43,대구광역시,합계,대구광역시,2374960,1094148,2.17
53,인천광역시,합계,인천광역시,2997410,1350912,2.22
67,광주광역시,합계,광주광역시,1419237,655433,2.17
73,대전광역시,합계,대전광역시,1442216,680261,2.12
79,울산광역시,합계,울산광역시,1103661,490690,2.25
85,세종특별자치시,합계,세종특별자치시,386525,160835,2.40
86,세종특별자치시,합계,세종특별자치시,386525,160835,2.40
87,경기도,합계,경기도,13630821,5978724,2.28


In [88]:
pop_info_df_final[~pop_info_df_final.duplicated()].shape

(288, 6)

In [89]:
pop_info_df_final.shape

(289, 6)

In [90]:
pop_info_df_final_1 = pop_info_df_final[~pop_info_df_final.duplicated()].copy()

In [91]:
for key, values in sido_convert.items():
    for v in values:
        pop_info_df_final_1.loc[pop_info_df_final_1.sido == v, 'sido'] = key

In [92]:
pop_info_df_final_1.sido.unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원특별자치도', '충청북도', '충청남도', '전북특별자치도', '전라남도',
       '경상북도', '경상남도', '제주특별자치도'], dtype=object)

In [93]:
for i in pop_info_df_final_1.columns:
    print(f'`{i}` ,')

`sido` ,
`sigungu` ,
`address` ,
`total_pop` ,
`total_family` ,
`pop_per_family` ,


In [94]:
pop_info_df_final_1

,sido,sigungu,address,total_pop,total_family,pop_per_family
0,서울특별시,합계,서울특별시,9386034,4469417,2.10
1,서울특별시,종로구,서울특별시 종로구,139417,72067,1.93
2,서울특별시,중구,서울특별시 중구,121312,64714,1.87
3,서울특별시,용산구,서울특별시 용산구,213151,107825,1.98
4,서울특별시,성동구,서울특별시 성동구,277361,133089,2.08
...,...,...,...,...,...,...
284,경상남도,거창군,경상남도 거창군,60047,30934,1.94
285,경상남도,합천군,경상남도 합천군,41203,23939,1.72
286,제주특별자치도,합계,제주특별자치도,675252,313155,2.16
287,제주특별자치도,제주시,제주특별자치도 제주시,491654,224433,2.19


In [95]:
sql = "INSERT INTO popInfo VALUES (%s, %s, %s, %s, %s, %s)"
pop_info_df_final_1.apply(lambda x : cur.execute(sql, x.tolist() ), axis=1)

0      1
1      1
2      1
3      1
4      1
      ..
284    1
285    1
286    1
287    1
288    1
Length: 288, dtype: int64

In [96]:
con.commit()

- - -
- area_info_df: 2023년도 총면적 조사

In [97]:
area_info_df = pd.read_csv('./../../../data/dataset_miniproject1/area_info.csv', encoding='cp949')
area_info_df.head(1)

,년(Annual),시도,시군구,면적,지번수
0,2023,전국,전국,100449356102,39631891


In [98]:
print(area_info_df.columns.tolist(), end='')

['년(Annual)', '시도', '시군구', '면적', '지번수']

In [99]:
area_info_df.columns = ['year', 'sido', 'sigungu', 'area', 'land_address_count']

In [100]:
area_info_df.loc[area_info_df.sigungu.str.contains('부산'), 'sigungu']

27      부산
32    부산진구
Name: sigungu, dtype: object

In [101]:
#area_info_df.sigungu.unique()
area_info_df.loc[area_info_df.sigungu.str.contains('\(계\)'), 'sigungu'] = area_info_df.loc[area_info_df.sigungu.str.contains('\(계\)'), 'sigungu'].apply(lambda x: x[:-3])

In [102]:
#area_info_df.loc[area_info_df.sigungu.str.contains('\(계\)'), 'sigungu']
area_info_df.sigungu.unique()

array(['전국', '서울', '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구',
       '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구',
       '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '부산',
       '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구',
       '연제구', '수영구', '사상구', '기장군', '대구', '수성구', '달서구', '달성군', '군위군', '인천',
       '미추홀구', '연수구', '남동구', '부평구', '계양구', '강화군', '옹진군', '광주', '광산구',
       '대전', '유성구', '대덕구', '울산', '울주군', '합계', '세종특별자치시', '경기', '수원시',
       '성남시', '의정부시', '안양시', '부천시', '광명시', '평택시', '동두천시', '안산시', '고양시',
       '과천시', '남양주시', '오산시', '시흥시', '군포시', '의왕시', '하남시', '용인시', '파주시',
       '이천시', '안성시', '김포시', '화성시', '광주시', '양주시', '포천시', '여주시', '연천군',
       '가평군', '양평군', '강원', '춘천시', '원주시', '강릉시', '동해시', '태백시', '속초시',
       '삼척시', '홍천군', '횡성군', '영월군', '평창군', '정선군', '철원군', '화천군', '양구군',
       '인제군', '고성군', '양양군', '충북', '청주시', '충주시', '제천시', '보은군', '옥천군',
       '영동군', '증평군', '진천군', '괴산군', '음성군', '단양군', '충남', '천안시', '공주시',
    

In [103]:
area_info_df.loc[(area_info_df.sigungu == area_info_df.sido) & (area_info_df.sigungu != '세종특별자치시'), 'sigungu'] = '합계'

In [104]:
area_info_df

,year,sido,sigungu,area,land_address_count
0,2023,전국,합계,100449356102,39631891
1,2023,서울,합계,605204030,907453
2,2023,서울,종로구,23913222,49139
3,2023,서울,중구,9959947,34540
4,2023,서울,용산구,21867577,44667
...,...,...,...,...,...
240,2023,경남,거창군,803956581,266331
241,2023,경남,합천군,983118399,328590
242,2023,제주,합계,1850273554,879102
243,2023,제주,제주시,978767267,523533


In [105]:
for key, values in sido_convert.items():
    for v in values:
        area_info_df.loc[area_info_df.sido == v, 'sido'] = key

In [106]:
area_info_df.sido.unique()

array(['전국', '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시',
       '울산광역시', '세종특별자치시', '경기도', '강원특별자치도', '충청북도', '충청남도', '전북특별자치도',
       '전라남도', '경상북도', '경상남도', '제주특별자치도'], dtype=object)

In [107]:
for i in area_info_df.columns:
    print(f'`{i}` ,')

`year` ,
`sido` ,
`sigungu` ,
`area` ,
`land_address_count` ,


In [113]:
sql = "INSERT INTO areaInfo VALUES (%s, %s, %s, %s, %s)"
area_info_df.apply(lambda x : cur.execute(sql, x.tolist() ), axis=1)

0      1
1      1
2      1
3      1
4      1
      ..
240    1
241    1
242    1
243    1
244    1
Length: 245, dtype: int64

In [114]:
con.commit()

In [110]:
con.close()

- - -
car_model_df: 전기차 모델별 주행거리

In [14]:
car_model_df = pd.read_csv('./../../../data/dataset_miniproject1/car_model.csv', encoding='cp949')
car_model_df.head(1)

,차종,"주행거리(1회 충전 시,km)"
0,2024 테슬라 모델S,514


In [15]:
car_model_df.columns = ['car_model', 'range_per_charge']

In [16]:
car_model_df

,car_model,range_per_charge
0,2024 테슬라 모델S,514
1,2024 테슬라 모델3,435
2,2024 대창모터스 다니고 W,186
3,2024 대창모터스 다니고 T,175
4,2024 대창모터스 다니고 R,186
...,...,...
64,2022 코란도 이모션,307
65,2022 DS 3 크로스백 E-텐스,260
66,2022 아우디 e-트론 GT,362
67,2022 아우디 RS e-트론 GT,336


In [17]:
for i in car_model_df.columns:
    print(f'`{i}` ,')

`car_model` ,
`range_per_charge` ,


In [18]:
#cur = con.cursor()
sql = "INSERT INTO carModel VALUES (%s, %s)"
car_model_df.apply(lambda x : cur.execute(sql, x.tolist() ), axis=1)
#cur.close()

0     1
1     1
2     1
3     1
4     1
     ..
64    1
65    1
66    1
67    1
68    1
Length: 69, dtype: int64

In [20]:
con.commit()

- - -
- charge_station_count: 전국 전기차 충전소 총 개수 (시도별)

In [115]:
charge_station_count_df = pd.read_csv('./../../../data/dataset_miniproject1/charge_station_count.csv', encoding='cp949')
charge_station_count_df.head(1)

,연도,시도,충전속도,개수
0,2023,서울,완속,44480


In [116]:
charge_station_count_df.columns = ['year', 'sido', 'charge_type', 'count']

In [117]:
for key, values in sido_convert.items():
    for v in values:
        charge_station_count_df.loc[charge_station_count_df.sido == v, 'sido'] = key

In [118]:
charge_station_count_df.sido.unique()

array(['서울특별시', '경기도', '인천광역시', '경상', '전라', '충청', '강원특별자치도', '제주특별자치도'],
      dtype=object)

In [119]:
for i in charge_station_count_df.columns:
    print(f'`{i}` ,')

`year` ,
`sido` ,
`charge_type` ,
`count` ,


In [120]:
sql = "INSERT INTO chargeStationCount VALUES (%s, %s, %s, %s)"
charge_station_count_df.apply(lambda x : cur.execute(sql, x.tolist() ), axis=1)

0     1
1     1
2     1
3     1
4     1
     ..
59    1
60    1
61    1
62    1
63    1
Length: 64, dtype: int64

In [121]:
con.commit()

- - -
- electric_car_regitrection_count_df: 전국 전기차 등록현황

In [122]:
electric_car_regitrection_count_df = pd.read_csv('./../../../data/dataset_miniproject1/electric_car_regitrection_count.csv', encoding='cp949')
electric_car_regitrection_count_df.head(1)

,연도,시도,개수
0,2023,서울,72937


In [123]:
electric_car_regitrection_count_df.columns = ['year', 'sido', 'count']

In [124]:
for key, values in sido_convert.items():
    for v in values:
        electric_car_regitrection_count_df.loc[electric_car_regitrection_count_df.sido == v, 'sido'] = key

In [125]:
for i in electric_car_regitrection_count_df.columns:
    print(f'`{i}` ,')

`year` ,
`sido` ,
`count` ,


In [126]:
electric_car_regitrection_count_df.sido.unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원특별자치도', '충청북도', '충청남도', '전북특별자치도', '전라남도',
       '경상북도', '경상남도', '제주특별자치도'], dtype=object)

In [127]:
sql = "INSERT INTO electricCarRegitrectionCount VALUES (%s, %s, %s)"
electric_car_regitrection_count_df.apply(lambda x : cur.execute(sql, x.tolist() ), axis=1)

0      1
1      1
2      1
3      1
4      1
      ..
97     1
98     1
99     1
100    1
101    1
Length: 102, dtype: int64

In [128]:
con.commit()

- - -
- electric_demand_df: 전국 전기차 월별 전력 예측 요구량

In [129]:
electric_demand_df = pd.read_csv('./../../../data/dataset_miniproject1/electric_demand.csv', encoding='cp949')
electric_demand_df.head(1)

,연,월,도,시군구,월별 수요량
0,2022,2,강원,강릉시,25262


In [130]:
electric_demand_df.columns = ['year', 'month', 'sido', 'sigungu', 'demended_amount_kW']

In [131]:
for key, values in sido_convert.items():
    for v in values:
        electric_demand_df.loc[electric_demand_df.sido == v, 'sido'] = key

In [132]:
electric_demand_df.sido.unique()

array(['강원특별자치도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시',
       '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전북특별자치도',
       '제주특별자치도', '충청남도', '충청북도'], dtype=object)

In [133]:
for i in electric_demand_df.columns:
    print(f'`{i}` ,')

`year` ,
`month` ,
`sido` ,
`sigungu` ,
`demended_amount_kW` ,


In [134]:
electric_demand_df

,year,month,sido,sigungu,demended_amount_kW
0,2022,2,강원특별자치도,강릉시,25262
1,2022,2,강원특별자치도,고성군,5930
2,2022,2,강원특별자치도,동해시,18122
3,2022,2,강원특별자치도,삼척시,12619
4,2022,2,강원특별자치도,속초시,23110
...,...,...,...,...,...
5216,2023,12,충청북도,제천시,15631
5217,2023,12,충청북도,증평군,4578
5218,2023,12,충청북도,진천군,23476
5219,2023,12,충청북도,청주시,216169


In [135]:
sql = "INSERT INTO electricDemand VALUES (%s, %s, %s, %s, %s)"
electric_demand_df.apply(lambda x : cur.execute(sql, x.tolist() ), axis=1)

0       1
1       1
2       1
3       1
4       1
       ..
5216    1
5217    1
5218    1
5219    1
5220    1
Length: 5221, dtype: int64

In [136]:
con.commit()